<a href="https://colab.research.google.com/github/LarissaCarrara/SINALIA/blob/main/SINALIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Instalar Framework de agentes do Google
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente:  Busca traduzir a frase para libras --- #
##########################################
def agente_buscador(frase):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
       Você é um **Guia de Libras**. Sua tarefa é **traduzir frases do português para Libras**, **explicar como fazer os gestos**, e **fornecer um link** para ver o sinal em ação.

Use a ferramenta `Google Search` com esta estratégia:
1.  **Primeiro, tente buscar o sinal em Libras para a frase COMPLETA** que o usuário pedir. Priorize resultados que ofereçam vídeos ou imagens claras de como fazer o sinal da frase.
2.  **Se não encontrar o sinal para a frase completa**, então **quebre a frase em palavras-chave** e busque o sinal em Libras e um link para **CADA UMA DESSAS PALAVRAS**. Priorize vídeos ou imagens para cada palavra. Lembre-se que a estrutura da Libras pode ser diferente do português, então os sinais das palavras individuais são um ótimo ponto de partida.

Sua resposta DEVE ser muito útil e incluir:
* A **frase original** em português.
* **SE o sinal da frase completa for encontrado:**
    * Uma **descrição detalhada e clara de como fazer os gestos** do sinal da frase (movimentos, posição das mãos, expressão facial, etc.).
    * **O link** para o vídeo válido e disponível ou imagem que mostra o sinal da frase.
* **SE o sinal da frase completa NÃO for encontrado (e você buscar por palavras):**
    * Informe que, como a frase inteira não foi encontrada, você fornecerá os sinais das palavras-chave.
    * Para CADA palavra-chave encontrada, forneça:
        * A **palavra original**.
        * Uma **descrição clara de como fazer os gestos** do sinal daquela palavra.
        * **O link** para o vídeo ou imagem que mostra o sinal daquela palavra.
* **Se não encontrar nenhum link ou descrição clara (nem para a frase, nem para palavras) após todas as tentativas**, diga de forma educada que não foi possível encontrar um recurso visual específico e sugira buscar por um instrutor de Libras para um aprendizado mais aprofundado.
        """,
        description="Agente para ensinar libras",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Frase para passar para libras: {frase}"

    resultado = call_agent(buscador, entrada_do_agente_buscador)
    return resultado

In [ ]:
frase = input("Qual palavra/frase você gostaria de aprender em libras?: ")
resultado = agente_buscador(frase)
print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
display(to_markdown(resultado))

Qual palavra/frase você gostaria de aprender em libras?: Oi, tudo bem? Bem vindo ao meu projeto! Espero que goste.

--- 📝 Resultado do Agente 1 (Buscador) ---



> Olá! Para te ajudar a traduzir a frase "Oi, tudo bem? Bem-vindo ao meu projeto! Espero que goste" para Libras, vou buscar os sinais correspondentes.
> 
> 
> Olá! Como não encontrei um único sinal que traduza toda a sua frase em Libras, vou te ajudar com os sinais de cada parte dela:
> 
> *   **Oi:**
> 
>     *   Feche a mão e deixe apenas o dedo indicador ereto.
>     *   Gire o dedo indicador para frente, como se estivesse acenando.
>     *   [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEKRO3Vp4J5qTshnYMO74pSeopCL6dF7v1Ak3IC1_-iBU3qW_BHXTzkywmLnSj-bkdwLxVTzMFnFqOicNR_Spu38hz1IH9Waf5hQGLi3PZfMNM9GvWx2gKrsCEiSz3DsLhko_1Htw==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEKRO3Vp4J5qTshnYMO74pSeopCL6dF7v1Ak3IC1_-iBU3qW_BHXTzkywmLnSj-bkdwLxVTzMFnFqOicNR_Spu38hz1IH9Waf5hQGLi3PZfMNM9GvWx2gKrsCEiSz3DsLhko_1Htw==)
> 
> *   **Tudo bem?:**
> 
>     *   Sinalize "bom" com a mão em frente ao corpo (mão espalmada com o polegar para cima).
>     *   Em seguida, faça o sinal de "positivo" (joia).
>     *   A expressão facial deve acompanhar a pergunta.
> 
>     *   [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFwJ2T04UyKXweDiiMVE0OZ5u5bF1e6omABswJwEEcthsh83b28sXPx8RrJhgbJNMbv_7trpadrZQeNKnyXQzg1cVNTAE6RFS6XFM-iUolLXp8uaxL8P41AAHYOCXenJUUE2zQu](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFwJ2T04UyKXweDiiMVE0OZ5u5bF1e6omABswJwEEcthsh83b28sXPx8RrJhgbJNMbv_7trpadrZQeNKnyXQzg1cVNTAE6RFS6XFM-iUolLXp8uaxL8P41AAHYOCXenJUUE2zQu)
> 
> *   **Bem-vindo:**
> 
>     *   Coloque uma mão na frente do corpo e a outra atrás.
>     *   Faça um movimento puxando as mãos para você, como se estivesse convidando alguém a entrar.
>     *   [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGrpane3oj0C85PeC2YKRNmM6I243vjjzNFRTO4xCA1WYD3cZzFo9V05SGBV7tkYPqwjhOWmQIMD4G7sAn1eu4KAD21at-8tG7_dAgAp3T2GFQH_wwa1dYI4Uv5g4vIaIGg_C8xNQ==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGrpane3oj0C85PeC2YKRNmM6I243vjjzNFRTO4xCA1WYD3cZzFo9V05SGBV7tkYPqwjhOWmQIMD4G7sAn1eu4KAD21at-8tG7_dAgAp3T2GFQH_wwa1dYI4Uv5g4vIaIGg_C8xNQ==)
> 
> *   **Projeto:**
> 
>     *   Existem diferentes sinais para "projeto" dependendo do contexto. Um deles é com as duas mãos em "L", movendo-as em pequenos círculos. Outra opção é usar as duas mãos espalmadas, com os dedos indicadores apontando para frente, e movê-las para frente e para trás alternadamente.
>     *   [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG02qK_oTrXP_p-2yvvPvKcbg8p2Y9vaZpZpmTrtorQfAZkp2PJed33dA4jtUV9Fpexgd6e55OttWSh4rPIQIwbRybtTuR6FaAEpT0h2JMPOWmymvX2R-Ci0Wsco5J14YTOr62Sgg==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG02qK_oTrXP_p-2yvvPvKcbg8p2Y9vaZpZpmTrtorQfAZkp2PJed33dA4jtUV9Fpexgd6e55OttWSh4rPIQIwbRybtTuR6FaAEpT0h2JMPOWmymvX2R-Ci0Wsco5J14YTOr62Sgg==)
>     *   [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEN1ym_CPVC4-_waF7wPbRPs6dUhpkT-C_fHSLgdQ6rp6uEOZgvXknmHZkRSAsJVowvdHY86pO7TMPHBAwXkxcl3QMg1bAJCOACWXy7DG0kC-d0zbTKEKcXxhvJ6Q0ZcUO2OZjhwA==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEN1ym_CPVC4-_waF7wPbRPs6dUhpkT-C_fHSLgdQ6rp6uEOZgvXknmHZkRSAsJVowvdHY86pO7TMPHBAwXkxcl3QMg1bAJCOACWXy7DG0kC-d0zbTKEKcXxhvJ6Q0ZcUO2OZjhwA==)
> 
> *   **Espero:**
> 
>     *   O sinal de "espero" pode ser traduzido como "ter esperança".
>     *   Há um vídeo no TikTok que mostra os sinais de "Espero e Esperança"
> 
>     *   [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGyirHzKaxKwrDZXgDVRVDHwj5N9SCW3Q31duj45R3vRBTfe2Bw9RdfOQyJKVxp2qicGUQO5xzbt2fwIvD2fb8jQ2t4x-IPwY1x18_mGoZSD8eOJTBe7_-JAmhVE3vJRfhDnZf0kPMs_JOnWrnngJLBX3M4fQq4YbGn](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGyirHzKaxKwrDZXgDVRVDHwj5N9SCW3Q31duj45R3vRBTfe2Bw9RdfOQyJKVxp2qicGUQO5xzbt2fwIvD2fb8jQ2t4x-IPwY1x18_mGoZSD8eOJTBe7_-JAmhVE3vJRfhDnZf0kPMs_JOnWrnngJLBX3M4fQq4YbGn)
> 
> *   **Goste:**
> 
>     *   Leve a mão dominante aberta para o peito, encostando a palma.
>     *   Faça movimentos circulares.
>     *   [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE4mNd1gKqJX3wOw6Dd36CtLk_eP3zuaW8y4oZC2XbSp1KGjrvZMtJkE3ly0KaO6Fd9iTuRv5nRVtTOnL8luSK4Gd-yuT7aH2byYRmaKQCafDrG1ZcmMKXjR1chNnnMiQcrVQ6E](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE4mNd1gKqJX3wOw6Dd36CtLk_eP3zuaW8y4oZC2XbSp1KGjrvZMtJkE3ly0KaO6Fd9iTuRv5nRVtTOnL8luSK4Gd-yuT7aH2byYRmaKQCafDrG1ZcmMKXjR1chNnnMiQcrVQ6E)
> 
> Espero que isso ajude! Se precisar de mais alguma coisa, é só perguntar. 😊
> 
